In [7]:
print(f"\n** RAG implementation of Llama2 using pdf **")


** RAG implementation of Llama2 using pdf **


In [4]:
import os 
from dotenv import load_dotenv
load_dotenv()
#OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
MODEL = "llama2"

In [5]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

#model.invoke("how to impress my boyfriend who likes business and build wealth and family ?")

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
#chain.invoke("Tell me the joke")

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Arun_Resume.pdf")
pages = loader.load_and_split()
#pages


[Document(page_content='Arun Raja Manjini  \narunraja08@gmail.com |  (361) 228 -0045 | Hillsboro, OR  | linkedin.com/in/arunrjm/  \n \nOBJECTIVE  \nAs a n Engineer  my goal is to immerse myself in cutting -edge technologies, methodologies, and tools \nemployed in  pre-silicon & post -silicon validation, thereby broadening my expertise,  and learning latest \nvalidation methodologies /tools , and becoming an integral part of a team delivering high -quality  product.  \n \nEXPERIENCE  \nSystem Validation Engineer , Intel Corporation, USA                              Aug 2021 – Present  \n▪ Contributed to the test plans to functionally validate Intel Scalable coherent fabric’s features routing \nand cache coherency in LLC and other IO devices.  \n▪ Collaborated with the IP design Architecture / SoC  Emulation teams to review and debug of functional \nscenarios to functional/ performance validation.  \n▪ Implemented/ develop ed Python scripts for debug script and individual test cases, int

In [8]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [9]:
chain = prompt| model | parser  
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [10]:
chain.invoke(
 {
    "context":"My name is Aaron.",
    "question":"What's my name ?"
 })

'Of course! Based on the context you provided, the answer to the question "What\'s my name?" is "Aaron."'

In [14]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [16]:
retriever = vectorstore.as_retriever()
retriever.invoke("Cache")

[Document(page_content='Arun Raja Manjini  \narunraja08@gmail.com |  (361) 228 -0045 | Hillsboro, OR  | linkedin.com/in/arunrjm/  \n \nOBJECTIVE  \nAs a n Engineer  my goal is to immerse myself in cutting -edge technologies, methodologies, and tools \nemployed in  pre-silicon & post -silicon validation, thereby broadening my expertise,  and learning latest \nvalidation methodologies /tools , and becoming an integral part of a team delivering high -quality  product.  \n \nEXPERIENCE  \nSystem Validation Engineer , Intel Corporation, USA                              Aug 2021 – Present  \n▪ Contributed to the test plans to functionally validate Intel Scalable coherent fabric’s features routing \nand cache coherency in LLC and other IO devices.  \n▪ Collaborated with the IP design Architecture / SoC  Emulation teams to review and debug of functional \nscenarios to functional/ performance validation.  \n▪ Implemented/ develop ed Python scripts for debug script and individual test cases, int

In [17]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,#array of documents
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [20]:
questions = [
    "where did he study?",
    "Whats the name ?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: where did he study?
Answer: The answer to your question can be found in the resume of Arun Raja Manjini. According to his resume, he studied at Texas A&M University in the United States for his Master's degree in Electrical Engineering, and at Pondicherry University in India for his Bachelor's degree in Electronics & Communication.



In [21]:
chain.batch([{"question": q} for q in questions])

['Based on the provided resume, Arun Raja Manjini studied at Texas A&M University in the United States for his Master of Science degree in Electrical Engineering, and at Pondicherry University in India for his Bachelor of Technology degree in Electronics & Communication.']